In [8]:
import requests
import pandas as pd
from datetime import datetime

# Base URL for the GDELT API
base_url = "https://api.gdeltproject.org/api/v2/doc/doc"

# Parameters for the query
params = {
    'query': 'climate change',
    'mode': 'artlist',  # Requesting a list of articles
    'maxrecords': 100,  # Limit to 100 articles
    'format': 'json',  # Response format
    'sort': 'datedesc',  # Sort by date descending
    'startdatetime': datetime(2022, 8, 17).strftime('%Y%m%d%H%M%S'),  # Start date in YYYYMMDDHHMMSS format
    'enddatetime': datetime(2023, 8, 24).strftime('%Y%m%d%H%M%S'),  # End date in YYYYMMDDHHMMSS format
}

# Make the request
response = requests.get(base_url, params=params)

# Check if the request was successful and content is not empty
if response.status_code == 200 and response.text:
    try:
        data = response.json()
        # Convert the articles into a DataFrame
        articles = pd.DataFrame(data['articles'])

        # Convert 'seendate' to a more readable format
        articles['seendate'] = pd.to_datetime(articles['seendate'], format='%Y%m%dT%H%M%SZ')

    except ValueError as e:
        print(f"Failed to parse JSON: {e}")
else:
    print(f"Failed to retrieve data or empty response: {response.status_code}")


In [9]:
articles

,url,url_mobile,title,seendate,socialimage,domain,language,sourcecountry
0,https://ampravda.ru/2023/08/24/0123838.html,,Почему жители Амурской области остались без же...,2023-08-24 01:00:00,https://ampravda.ru/files/articles-2/123838/lr...,ampravda.ru,Russian,Russia
1,https://www.zazoom.it/2023-08-24/gli-operai-de...,,Gli operai dei cantieri assediati dal caldo | ...,2023-08-24 01:00:00,https://www.bergamonews.it/photogallery_new/im...,zazoom.it,Italian,Italy
2,https://www.lawtimesnews.com/resources/legal-e...,,Law profession at a critical juncture says i...,2023-08-24 01:00:00,https://cdn-res.keymedia.com/cms/images/ca/119...,lawtimesnews.com,English,Canada
3,https://www.macrobusiness.com.au/2023/08/will-...,,Will China property black hole suck in Xi Jinp...,2023-08-24 01:00:00,https://api.macrobusiness.com.au/wp-content/up...,macrobusiness.com.au,English,Australia
4,https://www.saultstar.com:443/opinion/columnis...,https://www.saultstar.com/opinion/columnists/g...,Glavin : Canada is colluding in China global g...,2023-08-24 01:00:00,https://smartcdn.gprod.postmedia.digital/nexus...,saultstar.com,English,Canada
...,...,...,...,...,...,...,...,...
95,https://www.timesonline.com/story/news/nation/...,,March on Washington 60th Anniversary : Faces i...,2023-08-24 00:30:00,https://www.gannett-cdn.com/presto/2023/07/11/...,timesonline.com,English,United States
96,https://www.maritime-executive.com/article/off...,,Offshore Oil Workers Affected by Severe Heat Wave,2023-08-24 00:15:00,https://maritime-executive.com/media/images/ar...,maritime-executive.com,English,United States
97,https://www.wnd.com/2023/08/climate-brainwashi...,,Climate brainwashing … of first graders,2023-08-24 00:15:00,https://www.wnd.com/wp-content/uploads/2023/06...,wnd.com,English,United States
98,https://www.capital.fr/economie-politique/dans...,,"Dans le sud de la France , des caméras intelli...",2023-08-24 00:15:00,https://cap.img.pmdstatic.net/fit/https.3A.2F....,capital.fr,French,France


In [ ]:
from gdeltdoc import GdeltDoc

# Create an instance of the GdeltDoc class
gd = GdeltDoc()

# Define the query parameters
query = "climate change"
start_date = "2023-07-01"
end_date = "2023-08-01"

# Perform the query
results = gd.query(query=query, start_date=start_date, end_date=end_date)

# Convert the results to a pandas DataFrame
df = pd.DataFrame(results)

# Display the DataFrame
print(df.head())